<a href="https://colab.research.google.com/github/silvitreatment/ML_homework/blob/main/butterfly_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok pillow numpy pandas matplotlib -q

import tensorflow as tf
print(f"TensorFlow версия: {tf.__version__}")

TensorFlow версия: 2.19.0


In [1]:
import requests
import json
from pathlib import Path
import time
from PIL import Image
from io import BytesIO

DATA_DIR = Path("butterfly_dataset")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# 8 видов бабочек с их taxon_id
BUTTERFLIES = {
    "monarch": 48662,  # Данаида монарх
    "admiral": 49133,  # Адмирал
    "cabbage_white": 55626,  # Белянка репная
    "silvery_checkerspot": 49150,  # Геликонида диона
    "painted_lady": 48548,  # Репейница
    "tiger_swallowtail": 60551,  # Парусник главк
    "peacock": 207977,  # Дневной павлиний глаз
    "common_blue": 55641,  # Голубянка икар
}

# Сколько фото скачать для каждого вида
TARGET_COUNT = 150

def get_best_photo_url(photo_data):
    """
    Выбирает лучший URL фотографии из доступных размеров
    Порядок предпочтения: original > large > medium > small > square
    """
    # Пробуем получить самый большой размер
    if photo_data.get('original_url'):
        return photo_data['original_url']
    elif photo_data.get('large_url'):
        return photo_data['large_url']
    elif photo_data.get('medium_url'):
        return photo_data['medium_url']
    elif photo_data.get('small_url'):
        return photo_data['small_url']
    elif photo_data.get('url'):  # square (самый маленький)
        # Пытаемся преобразовать square в large
        url = photo_data['url']
        # Заменяем square на large в URL
        if 'square' in url:
            return url.replace('square', 'large')
        elif 'thumb' in url:
            return url.replace('thumb', 'large')
        return url
    return None


def download_butterfly_hd(species_name, taxon_id, target_count=150):
    # Создаем папку для вида
    output_dir = DATA_DIR / species_name
    output_dir.mkdir(parents=True, exist_ok=True)

    downloaded = 0
    page = 1
    per_page = 30  # 30 наблюдений на страницу

    # Проверяем, сколько фото уже скачано
    existing_files = list(output_dir.glob("*.jpg"))
    already_downloaded = len(existing_files)

    if already_downloaded >= target_count:
        print(f"Уже скачано: {already_downloaded}/{target_count}")
        return already_downloaded

    print(f"Уже есть: {already_downloaded} фото")
    print(f"Нужно скачать еще: {target_count - already_downloaded}")

    downloaded = already_downloaded

    while downloaded < target_count:
        print(f"\nСтраница {page}...")

        # URL для запроса наблюдений
        url = "https://api.inaturalist.org/v1/observations"

        # Параметры запроса
        params = {
            'taxon_id': taxon_id,
            'quality_grade': 'research',  # Только проверенные наблюдения
            'has[]': 'photos',  # Только с фотографиями
            'page': page,
            'per_page': per_page,
            'order_by': 'observed_on',  # Сортировка по дате
            'order': 'desc'  # Свежие сначала
        }

        try:
            # Делаем запрос к API
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()  # Проверяем на ошибки

            data = response.json()

            # Если нет результатов - выходим
            if not data.get('results'):
                print(f"  Нет результатов на странице {page}")
                break

            print(f"  Найдено наблюдений: {len(data['results'])}")

            # Обрабатываем каждое наблюдение
            for obs in data['results']:
                if downloaded >= target_count:
                    break

                # Проверяем, что есть фотографии
                if not obs.get('photos'):
                    continue

                # Берем первую фотографию из наблюдения
                photo = obs['photos'][0]

                # Получаем лучший URL фотографии
                photo_url = get_best_photo_url(photo)

                if not photo_url:
                    continue

                # Проверяем, не скачивали ли мы уже эту фотографию
                # Создаем имя файла на основе observation_id и photo_id
                obs_id = obs.get('id', 'unknown')
                photo_id = photo.get('id', 'unknown')
                filename = f"{species_name}_{obs_id}_{photo_id}.jpg"
                img_path = output_dir / filename

                # Если файл уже существует, пропускаем
                if img_path.exists():
                    downloaded += 1
                    continue

                try:
                    # Скачиваем фотографию
                    img_response = requests.get(photo_url, timeout=20)
                    img_response.raise_for_status()

                    # Открываем как изображение для проверки
                    img = Image.open(BytesIO(img_response.content))

                    # Проверяем размеры
                    width, height = img.size

                    # Пропускаем слишком маленькие фото
                    if width < 400 or height < 300:
                        # print(f"  Слишком маленькое: {width}x{height}")
                        continue

                    # Сохраняем фотографию
                    img.save(img_path, 'JPEG', quality=95, optimize=True)

                    downloaded += 1

                    # Выводим прогресс
                    print(f"  [{downloaded:3d}/{target_count}] {filename[:30]}... ({width}x{height})")

                    # Короткая пауза между скачиваниями
                    time.sleep(0.3)

                except Exception as img_error:
                    # print(f"  Ошибка скачивания фото: {img_error}")
                    continue

            # Переходим к следующей странице
            page += 1

            # Пауза между страницами
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"  Ошибка API на странице {page}: {e}")
            time.sleep(5)  # Пауза при ошибке
            continue
        except Exception as e:
            print(f"  Неожиданная ошибка: {e}")
            break

    # Финальный результат
    total = len(list(output_dir.glob("*.jpg")))
    print(f"\n✓ {species_name.upper()}: {total} фото скачано")

    return total


def main():
    total_downloaded = 0
    results = []

    # Скачиваем для каждого вида
    for species_name, taxon_id in BUTTERFLIES.items():
        count = download_butterfly_hd(species_name, taxon_id, TARGET_COUNT)
        total_downloaded += count
        results.append((species_name, count))

        # Пауза между видами (чтобы не перегружать API)
        if species_name != list(BUTTERFLIES.keys())[-1]:  # Если не последний вид
            time.sleep(3)

    total_size_mb = 0

    for species_name, count in results:
        species_dir = DATA_DIR / species_name
        if species_dir.exists():
            # Считаем размер папки
            size_bytes = sum(f.stat().st_size for f in species_dir.glob("*.jpg"))
            size_mb = size_bytes / (1024 * 1024)
            total_size_mb += size_mb

            print(f"{species_name:20s}: {count:3d} фото, {size_mb:6.1f} МБ")

    for item in DATA_DIR.iterdir():
        if item.is_dir():
            jpg_count = len(list(item.glob("*.jpg")))
            print(f"{item.name}/ - {jpg_count} фото")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nСкачивание прервано пользователем")
    except Exception as e:
        print(f"\nКритическая ошибка: {e}")
        import traceback

        traceback.print_exc()

Уже есть: 0 фото
Нужно скачать еще: 150

Страница 1...
  Найдено наблюдений: 30
  [  1/150] monarch_336390069_611280729.jp... (768x1024)
  [  2/150] monarch_336388847_611278865.jp... (768x1024)
  [  3/150] monarch_336387771_611276887.jp... (768x1024)
  [  4/150] monarch_336409337_611320046.jp... (768x1024)
  [  5/150] monarch_336372090_611244694.jp... (768x1024)
  [  6/150] monarch_336381032_611263798.jp... (768x1024)
  [  7/150] monarch_336413257_611328048.jp... (926x809)
  [  8/150] monarch_336418474_611335174.jp... (1024x968)
  [  9/150] monarch_336354836_611210867.jp... (747x747)
  [ 10/150] monarch_336334435_611170871.jp... (768x1024)
  [ 11/150] monarch_336326217_611168115.jp... (768x1024)
  [ 12/150] monarch_336408265_611317584.jp... (803x1024)
  [ 13/150] monarch_336305272_611111113.jp... (768x1024)
  [ 14/150] monarch_336304763_611110023.jp... (768x1024)
  [ 15/150] monarch_336304647_611109707.jp... (768x1024)
  [ 16/150] monarch_336304608_611109627.jp... (768x1024)
  [ 17/150

In [ ]:
# 1. Установка необходимых библиотек
!pip install tensorflow scikit-learn matplotlib seaborn pandas pillow -q

# 2. Импорт библиотек
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import yaml
from PIL import Image
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow версия: {tf.__version__}")
print(f"Используется GPU: {'Да' if tf.config.list_physical_devices('GPU') else 'Нет'}")
# Проверяем, есть ли датасет
data_dir = Path("/content/butterfly_dataset")
if data_dir.exists():
    print(f"Датасет найден в {data_dir}")

    # Подсчитываем количество изображений по классам
    class_counts = {}
    total_images = 0

    for species_folder in data_dir.iterdir():
        if species_folder.is_dir():
            species_name = species_folder.name
            images = list(species_folder.glob("*.jpg")) + list(species_folder.glob("*.jpeg")) + list(species_folder.glob("*.png"))
            count = len(images)
            class_counts[species_name] = count
            total_images += count

    print(f"Всего изображений: {total_images}")
    print("Количество по классам:")
    for species, count in class_counts.items():
        print(f"  {species}: {count} изображений")
else:
    print("Датасет не найден. Убедитесь, что он загружен в /content/butterfly_dataset/")

In [ ]:
def split_dataset_colab():
    data_dir = Path("/content/butterfly_dataset")
    output_dir = Path("/content/butterflies_dataset_split")

    if not data_dir.exists():
        print("Исходный датасет не найден!")
        return False

    # Удаляем старые разделенные данные если есть
    if output_dir.exists():
        shutil.rmtree(output_dir)
        print("Удалены старые разделенные данные")

    # Создаем структуру папок
    splits = ["train", "val", "test"]
    for split in splits:
        (output_dir / split).mkdir(parents=True, exist_ok=True)

    all_images = []
    all_labels = []
    species_names = []

    # Собираем все изображения и метки
    species_folders = sorted([f for f in data_dir.iterdir() if f.is_dir()])

    for species_idx, species_folder in enumerate(species_folders):
        species_name = species_folder.name
        species_names.append(species_name)

        images = list(species_folder.glob("*.jpg")) + \
                list(species_folder.glob("*.jpeg")) + \
                list(species_folder.glob("*.png"))

        for img_path in images:
            all_images.append((img_path, species_name))
            all_labels.append(species_idx)

    if len(all_images) == 0:
        print("Нет изображений для разделения")
        return False

    print(f"Всего найдено изображений: {len(all_images)}")

    # Преобразуем метки в numpy array для stratify
    labels_np = np.array(all_labels)

    # Разделение на train+val и test (85/15)
    train_val_idx, test_idx = train_test_split(
        range(len(all_images)),
        test_size=0.15,
        random_state=42,
        stratify=labels_np
    )

    train_val_images = [all_images[i] for i in train_val_idx]
    train_val_labels = [all_labels[i] for i in train_val_idx]
    test_images = [all_images[i] for i in test_idx]

    # Разделение train_val на train и val (~70/15)
    train_idx, val_idx = train_test_split(
        range(len(train_val_images)),
        test_size=0.176,  # 0.15 / 0.85 ≈ 0.176
        random_state=42,
        stratify=np.array(train_val_labels)
    )

    train_images = [train_val_images[i] for i in train_idx]
    val_images = [train_val_images[i] for i in val_idx]

    # Функция для копирования изображений
    def copy_images(images, split_name):
        for img_path, species_name in images:
            dest_dir = output_dir / split_name / species_name
            dest_dir.mkdir(exist_ok=True)
            shutil.copy(img_path, dest_dir / img_path.name)

    copy_images(train_images, "train")
    copy_images(val_images, "val")
    copy_images(test_images, "test")

    # Сохраняем названия классов
    with open(output_dir / "classes.txt", "w", encoding='utf-8') as f:
        for name in species_names:
            f.write(f"{name}\n")

    print(f"\nДатасет успешно разделен:")
    print(f"  Train: {len(train_images)} изображений")
    print(f"  Val: {len(val_images)} изображений")
    print(f"  Test: {len(test_images)} изображений")

    return True

In [ ]:
def train_model_colab(epochs=10, batch_size=32):
    train_dir = "/content/butterflies_dataset_split/train"

    if not os.path.exists(train_dir):
        print("Обучающие данные не найдены!")
        return None

    # Параметры
    img_size = 224

    # Подготовка данных
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    # Загрузка данных
    train_data = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    # Проверяем наличие валидационных данных
    val_dir = "/content/butterflies_dataset_split/val"
    val_data = None
    if os.path.exists(val_dir):
        val_data = val_datagen.flow_from_directory(
            val_dir,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False
        )

    print(f"Количество классов: {train_data.num_classes}")
    print(f"Обучающих изображений: {train_data.samples}")

    if val_data:
        print(f"Валидационных изображений: {val_data.samples}")

    # Создаем модель
    base_model = keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(img_size, img_size, 3)
    )

    # Замораживаем базовую модель
    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(train_data.num_classes, activation='softmax')
    ])

    # Компиляция
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy' if val_data else 'accuracy',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ModelCheckpoint(
            '/content/best_model.h5',
            monitor='val_accuracy' if val_data else 'accuracy',
            save_best_only=True,
            mode='max'
        )
    ]


    # Обучение
    if val_data:
        history = model.fit(
            train_data,
            epochs=epochs,
            validation_data=val_data,
            callbacks=callbacks,
            verbose=1
        )
    else:
        history = model.fit(
            train_data,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

    # Сохраняем финальную модель
    model.save('/content/butterfly_classifier.h5')
    print("Модель сохранена: /content/butterfly_classifier.h5")

    # Сохраняем метрики
    metrics = {
        'train_accuracy': float(history.history['accuracy'][-1]),
        'train_loss': float(history.history['loss'][-1])
    }

    if val_data and 'val_accuracy' in history.history:
        metrics['val_accuracy'] = float(history.history['val_accuracy'][-1])
        metrics['val_loss'] = float(history.history['val_loss'][-1])

    with open('/content/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)

    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    if val_data and 'val_accuracy' in history.history:
        axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[0].set_title('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    # Loss
    axes[1].plot(history.history['loss'], label='Train Loss')
    if val_data and 'val_loss' in history.history:
        axes[1].plot(history.history['val_loss'], label='Val Loss')
    axes[1].set_title('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('/content/training_history.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\nМетрики обучения:")
    for key, value in metrics.items():
        if 'accuracy' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:.4f}")

    return model

In [ ]:
def test_model_colab():
    """Тестирование модели в Google Colab"""
    model_path = "/content/butterfly_classifier.h5"

    if not os.path.exists(model_path):
        print("Модель не найдена! Сначала выполните обучение.")
        return

    # Загружаем модель
    model = keras.models.load_model(model_path)
    print("Модель загружена для тестирования")

    test_dir = "/content/butterflies_dataset_split/test"

    if not os.path.exists(test_dir):
        print("Тестовые данные не найдены!")
        return

    # Загружаем названия классов
    classes_file = "/content/butterflies_dataset_split/classes.txt"
    if os.path.exists(classes_file):
        with open(classes_file, "r", encoding='utf-8') as f:
            class_names = [line.strip() for line in f]
    else:
        # Определяем классы из структуры папок
        class_names = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])

    print(f"Классы: {class_names}")

    # Тестирование
    test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    test_data = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    print(f"Тестирование на {test_data.samples} изображениях")

    # Предсказания
    predictions = model.predict(test_data, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_data.classes

    # Метрики
    correct = np.sum(predicted_classes == true_classes)
    total = len(true_classes)
    accuracy = correct / total * 100

    print(f"\nРезультаты тестирования:")
    print(f"Точность: {accuracy:.2f}%")
    print(f"Правильно классифицировано: {correct}/{total}")

    # Сохраняем результаты
    test_metrics = {
        'test_accuracy': float(accuracy),
        'test_correct': int(correct),
        'test_total': int(total)
    }

    with open('/content/test_metrics.json', 'w') as f:
        json.dump(test_metrics, f, indent=2)

    # Отчет по классам
    print("\nОтчет по классам:")
    print(classification_report(true_classes, predicted_classes, target_names=class_names, digits=3))

    # Матрица ошибок
    cm = confusion_matrix(true_classes, predicted_classes)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('/content/confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()

    return accuracy

In [ ]:
def main_colab():
    if split_dataset_colab():
        print("Датасет успешно разделен")
    else:
        print("Ошибка при разделении датасета")
        return

    model = train_model_colab(epochs=10, batch_size=32)
    if model:
        print("Модель успешно обучена")
    else:
        print("Ошибка при обучении модели")
        return

    accuracy = test_model_colab()
    if accuracy:
        print(f"Тестирование завершено с точностью {accuracy:.2f}%")


    files_to_check = [
        '/content/butterfly_classifier.h5',
        '/content/best_model.h5',
        '/content/metrics.json',
        '/content/test_metrics.json',
        '/content/training_history.png',
        '/content/confusion_matrix.png'
    ]

    for file_path in files_to_check:
        if os.path.exists(file_path):
            size = os.path.getsize(file_path) / 1024 / 1024
            print(f"{os.path.basename(file_path)} ({size:.1f} MB)")


# Запуск главной функции
if __name__ == "__main__":
    main_colab()

Удалены старые разделенные данные
Всего найдено изображений: 1200

Датасет успешно разделен:
  Train: 840 изображений
  Val: 180 изображений
  Test: 180 изображений
Датасет успешно разделен
Found 840 images belonging to 8 classes.
Found 180 images belonging to 8 classes.
Количество классов: 8
Обучающих изображений: 840
Валидационных изображений: 180


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2583 - loss: 2.2402

27/27 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.2616 - loss: 2.2297 - val_accuracy: 0.6167 - val_loss: 1.1203
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5672 - loss: 1.2739

27/27 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.5679 - loss: 1.2709 - val_accuracy: 0.6667 - val_loss: 0.8945
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6564 - loss: 0.9862

27/27 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.6562 - loss: 0.9866 - val_accuracy: 0.7222 - val_loss: 0.7897
Epoch 4/10
13/27 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.6578 - loss: 0.8781

дальше нужно сайтик через streamlit, на который можно загрузить фотку, с помощью модели определить к какому виду она относится и выдать 5 похожих. код ниже не работает.

In [ ]:
!pip install streamlit pyngrok tensorflow pillow numpy pandas -q

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import os
import random

st.set_page_config(page_title="Классификатор бабочек", layout="wide")
st.title("Классификатор бабочек")
st.write("Загрузите фотографию бабочки для определения вида")

def load_class_names():
    try:
        paths = [
            '/content/butterflies_dataset_split/classes.txt',
            'butterflies_dataset_split/classes.txt',
            'classes.txt'
        ]

        for path in paths:
            if os.path.exists(path):
                with open(path, 'r', encoding='utf-8') as f:
                    classes = [line.strip() for line in f.readlines()]
                return classes
    except:
        pass

    return [
        "Monarch", "Admiral", "Cabbage White", "Silvery Checkerspot",
        "Painted Lady", "Tiger Swallowtail", "Peacock", "Common Blue"
    ]

CLASS_NAMES = load_class_names()

@st.cache_resource
def load_model():
    try:
        model_paths = [
            '/content/butterfly_classifier.h5',
            '/content/best_model.h5',
            'butterfly_classifier.h5',
            'best_model.h5'
        ]

        for path in model_paths:
            if os.path.exists(path):
                model = keras.models.load_model(path)
                st.sidebar.write(f"Модель загружена: {os.path.basename(path)}")
                return model

        st.sidebar.write("Модель не найдена")
        return None

    except Exception as e:
        st.sidebar.write(f"Ошибка: {str(e)[:100]}")
        return None

def preprocess_image(image):
    image = image.resize((224, 224))
    img_array = np.array(image)

    if len(img_array.shape) == 2:
        img_array = np.stack([img_array] * 3, axis=-1)
    elif img_array.shape[2] == 4:
        img_array = img_array[:, :, :3]

    if img_array.max() > 1:
        img_array = img_array / 255.0

    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def find_sample_images(class_name, num_samples=5):
    search_paths = [
        f"/content/butterflies_dataset_split/train/{class_name}",
        f"/content/butterflies_dataset_split/val/{class_name}",
        f"/content/butterflies_dataset_split/test/{class_name}",
        f"/content/butterfly_dataset/{class_name}",
        f"butterflies_dataset_split/train/{class_name}",
        class_name
    ]

    for path in search_paths:
        if os.path.exists(path) and os.path.isdir(path):
            images = []
            for file in os.listdir(path):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    images.append(os.path.join(path, file))

            if images:
                selected = random.sample(images, min(num_samples, len(images)))
                return selected

    return []

model = load_model()

with st.sidebar:
    st.header("Информация")

    if model:
        st.write("Модель загружена")
    else:
        st.write("Модель не загружена")

    st.write(f"Классифицирует {len(CLASS_NAMES)} видов:")
    for i, name in enumerate(CLASS_NAMES, 1):
        st.write(f"{i}. {name}")

uploaded_file = st.file_uploader("Выберите изображение бабочки", type=['jpg', 'jpeg', 'png'])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')

    col1, col2 = st.columns(2)

    with col1:
        st.image(image, caption="Загруженное изображение", use_column_width=True)

    with col2:
        if model:
            with st.spinner("Анализ..."):
                processed = preprocess_image(image)
                predictions = model.predict(processed, verbose=0)
                predicted_class = np.argmax(predictions[0])
                confidence = float(predictions[0][predicted_class])

                class_name = CLASS_NAMES[predicted_class]

                st.success(f"Вид: {class_name}")
                st.metric("Уверенность", f"{confidence:.1%}")

                st.write("Вероятности по классам:")
                import pandas as pd
                probs_df = pd.DataFrame({
                    'Класс': CLASS_NAMES,
                    'Вероятность': predictions[0]
                })
                probs_df = probs_df.sort_values('Вероятность', ascending=False)
                st.dataframe(probs_df, hide_index=True, use_container_width=True)
        else:
            st.error("Модель не загружена")

    if model:
        st.divider()
        st.subheader(f"Примеры бабочек вида {class_name}")

        sample_images = find_sample_images(class_name)

        if sample_images:
            cols = st.columns(5)
            for idx, img_path in enumerate(sample_images[:5]):
                with cols[idx]:
                    try:
                        sample_img = Image.open(img_path)
                        st.image(sample_img, use_column_width=True, caption=f"Пример {idx+1}")
                    except:
                        st.write("Ошибка загрузки")
        else:
            st.write("Примеры изображений не найдены")

    if st.button("Загрузить новое изображение"):
        st.rerun()

else:
    st.write("Загрузите изображение бабочки для классификации")

In [ ]:
from google.colab import files
import os

# Проверяем, есть ли уже модель
if not os.path.exists('/content/butterfly_classifier.h5'):
    print("Загрузите файл модели butterfly_classifier.h5")
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.endswith('.h5'):
            # Копируем модель
            !cp "{filename}" /content/butterfly_classifier.h5
            print(f"Модель сохранена: /content/butterfly_classifier.h5")
else:
    print("Модель уже существует")

In [ ]:
!pip install streamlit -q
!streamlit run /content/app/app.py --server.port 8501 --server.address 0.0.0.0

Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: /content/app/app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true 2>&1 &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.78.204:8501

  Stopping...


In [ ]:
!wget -qO- https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 | tee cloudflared > /dev/null
!chmod +x cloudflared
!./cloudflared tunnel --url http://localhost:8501

2026-01-27T21:13:39Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-27T21:13:39Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-27T21:13:42Z INF +--------------------------------------------------------------------------------------------+
2026-01-27T21:13:42Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-27T21:13:42Z INF |  https://type-reef-mia-roommates.trycloudflare.com    

In [ ]:
!pip install streamlit -q

In [ ]:
%%writefile /content/app.py
import streamlit as st
import numpy as np
from PIL import Image
import os
import random

st.set_page_config(page_title="Butterfly Classifier")
st.title("Butterfly Classifier")

CLASSES = [
    "Monarch", "Admiral", "Cabbage White", "Silvery Checkerspot",
    "Painted Lady", "Tiger Swallowtail", "Peacock", "Common Blue"
]

def load_model():
    model_path = "/content/butterfly_classifier.h5"
    if os.path.exists(model_path):
        try:
            import tensorflow as tf
            model = tf.keras.models.load_model(model_path)
            return model
        except:
            return None
    return None

model = load_model()

def find_examples(class_name, num=5):
    search_paths = [
        f"/content/butterflies_dataset_split/train/{class_name}",
        f"/content/butterflies_dataset_split/val/{class_name}",
        f"/content/butterfly_dataset/{class_name}",
    ]

    for path in search_paths:
        if os.path.exists(path):
            images = [f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            if images:
                selected = random.sample(images, min(num, len(images)))
                return [os.path.join(path, img) for img in selected]
    return []

def process_image(image):
    img = image.resize((224, 224))
    img_array = np.array(img)

    if len(img_array.shape) == 2:
        img_array = np.stack([img_array] * 3, axis=-1)
    elif img_array.shape[2] == 4:
        img_array = img_array[:, :, :3]

    if img_array.max() > 1:
        img_array = img_array / 255.0

    return np.expand_dims(img_array, axis=0)

uploaded_file = st.file_uploader("Upload image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert('RGB')

    col1, col2 = st.columns(2)

    with col1:
        st.image(image, use_container_width=True)

    with col2:
        if model:
            processed = process_image(image)
            predictions = model.predict(processed, verbose=0)[0]

            pred_idx = np.argmax(predictions)
            confidence = predictions[pred_idx]
            pred_class = CLASSES[pred_idx]

            st.write(f"Species: {pred_class}")
            st.write(f"Confidence: {confidence:.1%}")
        else:
            st.write("Model not loaded")

    if model:
        st.write("---")
        st.write(f"Examples of {pred_class}:")

        examples = find_examples(pred_class, 5)

        if examples:
            cols = st.columns(5)
            for idx, img_path in enumerate(examples[:5]):
                with cols[idx]:
                    try:
                        ex_img = Image.open(img_path)
                        st.image(ex_img, use_container_width=True)
                    except:
                        st.write(f"Image {idx+1}")
        else:
            st.write("No examples found")

Overwriting /content/app.py


In [ ]:
import subprocess
import threading
import time

def run_streamlit():
    subprocess.run([
        'streamlit', 'run', '/content/app.py',
        '--server.port', '8501',
        '--server.headless', 'true'
    ])

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(3)

print("App is running.")
print("Click the eye icon on the right panel and open port 8501.")

App is running.
Click the eye icon on the right panel and open port 8501.


In [ ]:
from google.colab.output import eval_js
import time

# Даем приложению время на запуск
time.sleep(5)

# Пробуем получить URL
try:
  url = eval_js("google.colab.kernel.proxyPort(8501)")
  print(f"Your app is running at: {url}")
except:
  print("Could not get the URL automatically. Trying alternative...")
  # Попробуем другой способ: возможно, нужно обновить страницу Colab
  print("Please refresh the Colab page (F5) and run the cell again.")

Your app is running at: https://8501-m-s-26eanoqveyf0l-c.us-central1-1.prod.colab.dev


In [ ]:
# Установка Streamlit (если еще не установлен)
!pip install streamlit -q

# Проверка модели
import os
if not os.path.exists('/content/butterfly_classifier.h5'):
    print("Модель butterfly_classifier.h5 не найдена!")
    print("Убедитесь, что модель находится в /content/")
else:
    print("Модель найдена")

# Создание файла app.py
with open('/content/app.py', 'w') as f:
    f.write('''import streamlit as st
import numpy as np
from PIL import Image
import os
import random

st.set_page_config(page_title="Butterfly Classifier")
st.title("Butterfly Classifier")

CLASSES = [
    "Monarch", "Admiral", "Cabbage White", "Silvery Checkerspot",
    "Painted Lady", "Tiger Swallowtail", "Peacock", "Common Blue"
]

def load_model():
    model_path = "/content/butterfly_classifier.h5"
    if os.path.exists(model_path):
        try:
            import tensorflow as tf
            model = tf.keras.models.load_model(model_path)
            return model
        except:
            return None
    return None

model = load_model()

def find_examples(class_name, num=5):
    search_paths = [
        f"/content/butterflies_dataset_split/train/{class_name}",
        f"/content/butterflies_dataset_split/val/{class_name}",
        f"/content/butterfly_dataset/{class_name}",
    ]

    for path in search_paths:
        if os.path.exists(path):
            images = [f for f in os.listdir(path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
            if images:
                selected = random.sample(images, min(num, len(images)))
                return [os.path.join(path, img) for img in selected]
    return []

def process_image(image):
    img = image.resize((224, 224))
    img_array = np.array(img)

    if len(img_array.shape) == 2:
        img_array = np.stack([img_array] * 3, axis=-1)
    elif img_array.shape[2] == 4:
        img_array = img_array[:, :, :3]

    if img_array.max() > 1:
        img_array = img_array / 255.0

    return np.expand_dims(img_array, axis=0)

uploaded_file = st.file_uploader("Upload image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")

    col1, col2 = st.columns(2)

    with col1:
        st.image(image, use_container_width=True)

    with col2:
        if model:
            processed = process_image(image)
            predictions = model.predict(processed, verbose=0)[0]

            pred_idx = np.argmax(predictions)
            confidence = predictions[pred_idx]
            pred_class = CLASSES[pred_idx]

            st.write(f"Species: {pred_class}")
            st.write(f"Confidence: {confidence:.1%}")
        else:
            st.write("Model not loaded")

    if model:
        st.write("---")
        st.write(f"Examples of {pred_class}:")

        examples = find_examples(pred_class, 5)

        if examples:
            cols = st.columns(5)
            for idx, img_path in enumerate(examples[:5]):
                with cols[idx]:
                    try:
                        ex_img = Image.open(img_path)
                        st.image(ex_img, use_container_width=True)
                    except:
                        st.write(f"Image {idx+1}")
        else:
            st.write("No examples found")
''')

print("Файл app.py создан")

# Запуск через Cloudflared (не требует аккаунта)
print("\nЗапускаем Cloudflared...")

!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# Запускаем Streamlit в фоне
import subprocess
import threading
import time

def run_streamlit():
    subprocess.run([
        'streamlit', 'run', '/content/app.py',
        '--server.port', '8501',
        '--server.headless', 'true',
        '--server.address', '0.0.0.0'
    ])

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)

# Запускаем Cloudflared и получаем ссылку
import re

process = subprocess.Popen(
    ['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8501'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

print("Ждем запуска туннеля... (30-60 секунд)")

start_time = time.time()
url = None

while time.time() - start_time < 60 and url is None:
    line = process.stdout.readline()
    if line:
        print(line.strip())
        if 'https://' in line and 'trycloudflare.com' in line:
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                break
    time.sleep(0.1)

if url:
    print(f"\nВаше приложение доступно по ссылке:")
    print(f"{url}")
    print("\nСкопируйте эту ссылку и откройте в браузере.")
else:
    print("\nНе удалось получить ссылку автоматически.")
    print("Попробуйте открыть вручную:")
    print("1. Нажмите Ctrl+C в этой ячейке")
    print("2. Запустите команду: !streamlit run /content/app.py --server.port 8501")
    print("3. Обновите страницу Colab (F5)")
    print("4. Попробуйте найти иконку глаза в правой панели")

# Сохраняем ссылку в файл
if url:
    with open('/content/link.txt', 'w') as f:
        f.write(url)

Модель найдена
Файл app.py создан

Запускаем Cloudflared...
Ждем запуска туннеля... (30-60 секунд)
2026-01-28T02:16:33Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-28T02:16:33Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-28T02:16:36Z INF +--------------------------------------------------------------------------------------------+
2026-01-28T02:16:36Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to 